# Key Generation

In [6]:
from umbral import SecretKey, Signer, encrypt, decrypt_original, generate_kfrags, reencrypt, decrypt_reencrypted, VerifiedCapsuleFrag

In [2]:
alices_secret_key = SecretKey.random()
alices_public_key = alices_secret_key.public_key()

# alices_signing_key = SecretKey.random()
# alices_signer = Signer(alices_signing_key)
# alices_verifying_key = alices_signing_key.public_key()

bobs_secret_key = SecretKey.random()
bobs_public_key = bobs_secret_key.public_key()

# Encryption


In [3]:
plaintext = b'Proxy Re-encryption is cool!'
capsule, ciphertext = encrypt(alices_public_key, plaintext)

#decrypt with alice private key
cleartext = decrypt_original(alices_secret_key, capsule, ciphertext)
print(str(cleartext, 'utf-8'))

Proxy Re-encryption is cool!


# Re-Encryption Key Fragments

In [4]:
user_signing_key = SecretKey.random()
user_signer = Signer(user_signing_key)

kfrags = generate_kfrags(delegating_sk=alices_secret_key,
                         receiving_pk=bobs_public_key,
                         signer=user_signer,
                         threshold=2,
                         shares=2)

print(type(kfrags))
print(len(kfrags)) 
print(kfrags[0])

<class 'list'>
2
VerifiedKeyFrag:7e10ab22fb74e47b


# Re-Encryption

In [8]:
cfrags = list()
for kfrag in kfrags:
    print(kfrag)
    cfrag = reencrypt(capsule=capsule, kfrag=kfrag)
    cfrags.append(cfrag)
    
    print(cfrag)
    print(cfrag.__bytes__())
    new_cfrag = VerifiedCapsuleFrag.from_verified_bytes(cfrag.__bytes__())
    print(new_cfrag)

VerifiedKeyFrag:7e10ab22fb74e47b
VerifiedCapsuleFrag:0208a8b69d811b55
b'\x02\x08\xa8\xb6\x9d\x81\x1bU\xed\xb8b\x14\xc8\xa4\xa8\x9b\xebqj0\r:\xb6\x1a\xc7\xfe`\x1b\xbf\xb4\x17\x11/\x02\xa5v\x04N\xa1\xb8{\x1e\x05\x94\x1b\xd1$;m\x99\xc9\xecE\xfe\x88\xc51\xb9\xe1\x1bE\r\x1dt@\xe9~\x10\xab"\xfbt\xe4{\x15\xc1y\x83\x19[:\x8f\xffs\xbf}Of\x0f`\xcdt\xaf\xb8K\x00\x07\x1a\x03c@\x85\xd1T\xdf\xcc\x97m\x8d\xe1\xc9\xa2FT[\xc7\xd7\xe6\xfab\x08\xfe\xc6\xa7\'_\xf9\x06\x802\xb3\x02\x06c\x90\xe9x\xff\xdb\xf1M6\xd6\x03\xb1\xab\r\xaa\x01\xd7\xcc\xc1\xaf\x85\xf0U\xb6\xe7\xb0\xf4k\x1a5Y\x034\x84j$\x0b\x8f\x17H\x96\x9eW\xb9\xe1/\xd9U\x80\x9c\xcf\x98\x9b6@\x18Q\x97\xf9\x9f/\x87y\x06\x03{A\x1e\xd7\x07\n_\xef\xe0P\xc7F\xf8\xafV-\xbf=\xc62\xce]y\xe6\xdc\x02c{3\n\x93z\x02R\xb7\xe9\xf5\x1a*\x82\xab\xaap\x9c\x1c\xd8f\x94_R\xb9Es|\xa1H\'lK\x92\xba~\xa2\xff\x9c`ME\xfem\xba5\xd0\x81\xa9K\x90\x8c\xbf\xecn:\xaeQ)I\xff\xc2\x9f\x8b\x8e\xd8\x1b/a+\x89F\xd4E\xd1\x1b\xf4\xaaF\x8d\xbb\x05\xf1>\xb4\\\x05z>\xb0\xcd\x97\x14\xd9\x88\

# Decryption by Blob

In [6]:
bob_cleartext = decrypt_reencrypted(receiving_sk=bobs_secret_key,
                                        delegating_pk=alices_public_key,
                                        capsule=capsule,
                                        verified_cfrags=cfrags,
                                        ciphertext=ciphertext)
assert bob_cleartext == plaintext
print(str(bob_cleartext, 'utf-8'))

Proxy Re-encryption is cool!
